In [33]:
import torch
import gymnasium as gym
from Qlearning_agent.Qlearning import QLearner, train_agent_alpha
import gymnasium_env
import numpy as np
import matplotlib.pyplot as plt
from gymnasium_env.wrappers.probability import SkipMoveWrapper

In [34]:
from gymnasium.envs.registration import register

try:
    register(
        id="gymnasium_env/GridWorld-v0",
        entry_point="gymnasium_env.envs:GridWorldEnv",
    )
    print("Environment 'gymnasium_env/GridWorld-v0' registered successfully.")
except gym.error.AlreadyRegistered:
    print("Environment 'gymnasium_env/GridWorld-v0' already registered.")

Environment 'gymnasium_env/GridWorld-v0' registered successfully.


In [35]:
env = gym.make("gymnasium_env/GridWorld-v0", render_mode="human")


In [36]:
max_epsilon = 1
min_epsilon = 0.01
decay = 0.005
train_episodes = 2000
test_episodes = 100
max_steps = 500

In [37]:
n_actions = env.action_space.n
agent = QLearner(n_actions=n_actions)
rewards, epsilons = train_agent_alpha(env, agent, max_steps, train_episodes, min_epsilon, max_epsilon, decay)
print("Training score over time: " + str(sum(rewards)/train_episodes))
plt.plot(rewards)
plt.title("Learning Curve Comparison")
plt.xlabel("Episodes")
plt.ylabel("Total Reward")
plt.legend()
plt.grid(True)
plt.show()

Training Progress:   0%|          | 0/2000 [00:00<?, ?it/s]


TypeError: unhashable type: 'dict'

In [38]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25_000)

ModuleNotFoundError: No module named 'torch._C'